In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
import numpy as np
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


load_dotenv()


False

In [3]:
# Load pdf files in the local directory
def load_and_split_text(pdf_path):
    loader = PyPDFDirectoryLoader(pdf_path)

    # docs_before_split = loader.load()


    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size = 512,
    #     chunk_overlap  = 100,
    # )
    # docs_after_split = text_splitter.split_documents(docs_before_split)

    # return docs_after_split
    doc_pages = loader.load()
    return doc_pages
    

In [4]:
import re

def extract_substring_index(text, start_marker, end_marker):
    start_index = text.index(start_marker) + len(start_marker)
    end_index = text.index(end_marker, start_index)
    return text[start_index:end_index]

In [5]:

def get_metadata(text):
    AMBITO ='Ámbito Geográfico'
    INFORMACION='Información Detallada'

    document_tags = ['Referencia','Organismo','Sector','Subsector',
                    AMBITO,'Tipo','Destinatarios','Plazo de solicitud']

    tagIndex = 0
    metadata = {}
    metadataInText=""
    while tagIndex < len(document_tags)-1:
        start = document_tags[tagIndex]
        end = document_tags[tagIndex+1]
        if(start=='Ámbito Geográfico'):
            metadata[start]=extract_substring_index(text,start,end).replace(AMBITO,'').replace(INFORMACION,'').strip()
        else:
            metadata[start]=extract_substring_index(text,start,end).strip()
        #metadataInText = metadataInText+", "+start+" es "+metadata[start]
        tagIndex+=1
            
        
    #return [ metadata, metadataInText ]
    return metadata

In [6]:
from pathlib import Path
from urllib.parse import urlparse

import requests
def download_file(url,output_path,filename):
    response = requests.get(url)
    if response.status_code == 200:        
        with open(output_path+"/"+filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {url}")

In [7]:

from uuid import uuid4


def download_linked_files(page, output_path):
    urls=[]
    if "/Annots" in page:
        for annot in page["/Annots"]:
            annotObj = annot.get_object()
            if("/A" in annotObj):
                uri = annotObj.get("/A").get("/URI")
                if uri is not None:
                    print("[+] URL Found:", uri)
                    urls.append(uri)
    
    if(not os.path.exists(output_path)):
        os.makedirs(output_path)
    for url in urls:
        download_file(url, output_path, str(uuid4())+".pdf")             

In [8]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="jaimevera1107/all-MiniLM-L6-v2-similarity-es",
    #model_name="jinaai/jina-embeddings-v2-base-es",
    model_kwargs={'device':'cpu', 'trust_remote_code': True}, 
    encode_kwargs={'normalize_embeddings': False, 'attn_implementation': "eager"},
)

In [9]:
import shutil
def remove_foler(folder):
    try:
        shutil.rmtree(folder)
    except FileNotFoundError as e:                        
        print('The folder does not exist')

In [10]:
remove_foler('./ayudas/texto/Guia de Ayudas Sector Comercio.pdf')
#remove_foler('./db_subvenciones')

In [ ]:
import PyPDF2
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import chromadb
from chromadbx import UUIDGenerator
import os
from urllib.parse import urlparse
from chromadb.utils import embedding_functions


# Cargar modelo de embedding
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Crear una colección en ChromaDB

pathToMetadata = './ayudas/metadatos'
pathToText = './ayudas/texto'

# Función para procesar un PDF
def process_pdf(pdf_path):
    all_pages = []
    limit=0
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)        
        for page_num in range(len(reader.pages)):
            if limit == 10:
                break
            summary_page = reader.pages[page_num]
            text = summary_page.extract_text().replace("\n"," ")
            
            if (text.find("Ayudas e incentivos (detalle)") > -1):
                a = urlparse(pdf_path)
                output_dir = pathToText+"/"+os.path.basename(a.path)+"/"+"Page_"+str(page_num)
                
                #Get metadata from page
                """  metadata = get_metadata(text)
                page_metadata = metadata[0]
                page_metadataInText = metadata[1] """
                
                extra_metadata = get_metadata(text)
                download_linked_files(summary_page, output_dir)
    
                pages = load_and_split_text(output_dir)
                
                if(len(pages) > 0):


                    for page in pages:
                        page.metadata = page.metadata = {**page.metadata, **extra_metadata}
                all_pages=all_pages+pages
                limit+=1
    return all_pages            

# Procesar todos los PDFs en una carpeta
import os
limit = 1
allDocs = []

for file in os.listdir(pathToMetadata):
    if file.endswith(".pdf"):
        docs = process_pdf(os.path.join(pathToMetadata, file))
        allDocs= allDocs+docs

db = Chroma.from_documents(allDocs, huggingface_embeddings, persist_directory="./db_subvenciones",collection_name='subvenciones')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=71572&fichero=
Downloaded cc0aa7f6-8170-491d-a741-dfd694da388b.pdf
Downloaded ff834b4b-6930-4a6f-8d9c-3b8a15e96b76.pdf
Downloaded 26f14155-bfcf-43b5-9340-fbff635e6b86.pdf
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=111733&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=111733&fichero=
Downloaded 6159ffc7-162b-43f7-b09d-219f0b4d0623.pdf
Downloaded 0af2279c-be55-46f9-be87-832d6b11d14e.pdf
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=112431&fichero=
[+] URL Found: https://wapis.ipyme.org/servicioayudas/ayudas/detalle?id=112431&fichero=
Downloaded 5895e730-b8a3-4b5c-9c22-64f7db77c377.pdf
Downloaded 294bec5f-69cb-4dfa-8504-7b7f178f3933.pdf
[+] URL Found: https://

In [ ]:
import pprint
client = chromadb.PersistentClient('./db_subvenciones')
collection = client.get_collection('subvenciones')
results = collection.query(
    query_texts=["Ahorro energético"],
   # where={"Ámbito Geográfico": "Extremadura"}, # Chroma will embed this for you
    n_results=5 # how many results to return
)
pprint.pp(results)

{'ids': [['cb67dc41-444d-4ef9-98ac-d111b5ef0a6d',
          '168191b0-a915-49a5-a5d4-1370def8eb4c',
          'c516a693-96c8-45f0-a219-d7cf28e13830',
          'ad75e9ed-654a-4d59-902d-8909a9ec7319',
          '06103875-ca66-431f-8c15-233b10addd64']],
 'embeddings': None,
 'documents': [['NÚMERO 198 \n'
                'Jueves 10 de octubre de 2024\n'
                '49090\n'
                '  a)  Instancia solicitud debidamente cumplimentada en todos '
                'sus apartados y firmada elec-\n'
                'trónicamente, conforme al modelo del anexo I, que estará '
                'accesible en el apartado \n'
                'Solicitud de la ficha correspondiente y deberá presentarse en '
                'formato electrónico (pdf).\n'
                '    En el caso de comunidades de bienes y otras agrupaciones '
                'sin personalidad jurídica \n'
                'propia, deberán hacerse constar expresamente los compromisos '
                'de ejecución asu